<a href="https://colab.research.google.com/github/nazirumar/NLP/blob/main/Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Applying Text Similarity
Functions**

**Problem**

We will have multiple tables in the database, and sometimes there won’t
be a common “ID” or “KEY” to join them – scenarios like the following:
• Customer information scattered across multiple tables
and systems.
• No global key to link them all together.
• A lot of variations in names and addresses.


**Solution**

This can be solved by applying text similarity functions on the
demographic’s columns like the first name, last name, address, etc. And
based on the similarity score on a few common columns, we can decide
either the record pair is a match or not a match.




**Step 3A-1 Read and understand the data**

we need data first

import package

In [1]:
!pip install recordlinkage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.5/926.5 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.4 MB/s eta 0:00:00


In [8]:
import recordlinkage
#For this demo let us use the inbuilt dataset from recordlinkage library
#import data set from
from recordlinkage.datasets import load_febrl1


In [5]:
#create a dataframe - dfa
dfA = load_febrl1()
dfA.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933


**Step 3A-2 Blocking**

In [21]:
indexer = recordlinkage.Index()
indexer.block(left_on='given_name')
pairs = indexer.index(dfA)
print(len(pairs))

2082


**Step 3A-3 Similarity matching and scoring**

In [22]:
# This cell can take some time to compute.
compare_cl = recordlinkage.Compare()
compare_cl.string('given_name', 'given_name',method='jarowinkler',
label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler',
label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1',method='jarowinkler',
label='address_1')
features = compare_cl.compute(pairs, dfA)
features.sample(5)

,,given_name,surname,date_of_birth,suburb,state,address_1
rec_id_1,rec_id_2,,,,,,
rec-84-org,rec-166-org,1.0,0.428571,0,0,0,0.383333
rec-319-org,rec-319-dup-0,1.0,1.000000,1,1,1,0.000000
rec-151-org,rec-94-dup-0,1.0,0.577778,0,0,0,0.598214
rec-470-dup-0,rec-180-org,1.0,0.000000,0,0,0,0.460101
rec-122-dup-0,rec-248-org,1.0,0.600000,0,0,1,0.640491


**Step 3A-4 Predicting records match or do not match using
ECM – classifier**

In [23]:
# select all the features except for given_name since its our blocking key
features1 = features[['suburb','state','surname','date_of_birth','address_1']]
# Unsupervised learning – probabilistic
ecm = recordlinkage.ECMClassifier()
result_ecm = ecm.learn((features1).astype(int),return_type ='series')
result_ecm

/usr/local/lib/python3.10/dist-packages/recordlinkage/base.py:899: UserWarning: learn is deprecated, ECMClassifier.fit_predict instead
  warnings.warn("learn is deprecated, {}.fit_predict "


rec_id_1       rec_id_2     
rec-183-dup-0  rec-122-org      0
rec-248-org    rec-122-org      0
               rec-183-dup-0    0
rec-122-dup-0  rec-122-org      1
               rec-183-dup-0    0
                               ..
rec-252-org    rec-252-dup-0    1
rec-48-dup-0   rec-48-org       1
rec-298-dup-0  rec-298-org      1
rec-282-org    rec-282-dup-0    1
rec-327-org    rec-411-org      0
Name: classification, Length: 2082, dtype: int64

**Step 3B-1 Read and understand the data**

In [24]:
from recordlinkage.datasets import load_febrl4
dfA, dfB = load_febrl4()
dfA.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688


In [25]:
dfB.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884


**Step 3B-2 Blocking – to reduce the comparison window
and creating record pairs**

This is the same as explained previously, considering the given_name as a
blocking index:

In [28]:
indexer = recordlinkage.Index()
indexer.block(on='given_name')
pairs = indexer.index(dfA, dfB)

**Step 3B-3 Similarity matching**

In [30]:
compare_cl = recordlinkage.Compare()
compare_cl.string('given_name', 'given_name',method='jarowinkler',
label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler',
label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1',method='jarowinkler',
label='address_1')
features = compare_cl.compute(pairs, dfA, dfB)
features.head(10)

given_name   surname  date_of_birth  suburb  \
rec_id_1     rec_id_2                                                      
rec-1070-org rec-3024-dup-0         1.0  0.436508              0       0   
             rec-2371-dup-0         1.0  0.490079              0       0   
             rec-4652-dup-0         1.0  0.490079              0       0   
             rec-4795-dup-0         1.0  0.000000              0       0   
             rec-1314-dup-0         1.0  0.000000              0       0   
rec-2371-org rec-3024-dup-0         1.0  0.527778              0       0   
             rec-2371-dup-0         1.0  1.000000              1       1   
             rec-4652-dup-0         1.0  0.500000              0       0   
             rec-4795-dup-0         1.0  0.527778              0       0   
             rec-1314-dup-0         1.0  0.527778              0       0   

                             state  address_1  
rec_id_1     rec_id_2                          
rec-1070-org rec-3024-dup-0      1   0.000000  
             rec-2371-dup-0      0   0.715873  
             rec-4652-dup-0      0   0.645604  
             rec-4795-dup-0      1   0.552381  
             rec-1314-dup-0      1   0.618254  
rec-2371-org rec-3024-dup-0      0   0.000000  
             rec-2371-dup-0      1   1.000000  
             rec-4652-dup-0      1   0.635684  
             rec-4795-dup-0      0   0.411111  
             rec-1314-dup-0      0   0.672222

**Step 3B-4 Predicting records match or do not match using
ECM – classifier**

In [31]:
# select all the features except for given_name since its our blocking key
features1 = features[['suburb','state','surname','date_of_birth',
'address_1']]
# unsupervised learning - probablistic
ecm = recordlinkage.ECMClassifier()
result_ecm = ecm.learn((features1).astype(int),return_type =
'series')
result_ecm

/usr/local/lib/python3.10/dist-packages/recordlinkage/base.py:899: UserWarning: learn is deprecated, ECMClassifier.fit_predict instead
  warnings.warn("learn is deprecated, {}.fit_predict "


rec_id_1      rec_id_2      
rec-1070-org  rec-3024-dup-0    0
              rec-2371-dup-0    0
              rec-4652-dup-0    0
              rec-4795-dup-0    0
              rec-1314-dup-0    0
                               ..
rec-4528-org  rec-4528-dup-0    1
rec-4887-org  rec-4887-dup-0    1
rec-4350-org  rec-4350-dup-0    1
rec-4569-org  rec-4569-dup-0    1
rec-3125-org  rec-3125-dup-0    1
Name: classification, Length: 77249, dtype: int64